<a href="https://colab.research.google.com/github/Rheddes/recsys-twitter/blob/master/recsys_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Necessary imports & definitions

Copy files from drive to local disk, not necessary it is also possible to work directly from drive.

In [0]:
# !cp ./drive/My\ Drive/RecSys/train_updated.tsv train_updated.tsv
# !cp ./drive/My\ Drive/RecSys/sample.tsv sample.tsv 

Set train file variable to correct path

In [0]:
train_file = './drive/My Drive/RecSys/sample.tsv'

## Install transformers (for BERT models)


In [0]:
# !pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers

## Nvidia stats & info

In [2]:
# !nvcc --version
!nvidia-smi

Sun Apr 26 14:59:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Imports

In [0]:
import pandas as pd
import numpy as np
import csv
import math
import torch
import gc
import transformers as ppb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

## Tell pytorch to use cuda if available

In [11]:
use_cuda = True

print("Cuda is available: ", torch.cuda.is_available())
device = torch.device("cuda:0" if use_cuda and torch.cuda.is_available() else "cpu")

print("using device: ", device)

Cuda is available:  True
using device:  cuda:0


## Load pretrained models

In [13]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-multilingual-cased')

## Want BERT instead of distilBERT? Uncomment the following line:
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-multilingual-cased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights, do_lower_case=False)
model = model_class.from_pretrained(
    pretrained_weights,
    output_attentions = False,
    output_hidden_states = True,
)
model.eval()

print('done')

done


# Read the desired dataset

This piece of code can be used to read the desired dataset into memory as a Pandas dataframe.

In [5]:
all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",
                "tweet_type", "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",
                "engaged_with_user_following_count", "engaged_with_user_is_verified",
                "engaged_with_user_account_creation", "enaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count",
                "enaging_user_is_verified", "enaging_user_account_creation", "engagee_follows_engager",
                "reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]
                
dataset = pd.read_csv(train_file, delimiter="\x01", encoding='utf-8', header=None)
dataset.columns = all_features
print("done")

done


## Preprocessing

Parsing text_tokens column into numpy vector and padding them to the same size with zeroes so that they can be imported as tensors.

In [0]:
dataset = dataset[['text_tokens','like_timestamp']]

dataset.text_tokens = dataset.text_tokens.apply(lambda x: np.fromstring(x, dtype=int, sep="\t"))
dataset.like_timestamp = dataset.like_timestamp.apply(lambda x: 0 if math.isnan(x) else 1)

Padding text token vectors to same length

In [7]:
max_len = 0
for i in dataset.text_tokens.values:
  if len(i) > max_len:
      max_len = len(i)
print(max_len)

dataset.text_tokens = [np.concatenate([i, np.zeros(max_len-len(i), dtype=int)]) for i in dataset.text_tokens.values]

487


### Create dataloader

We can't use all data at the same time so we have to work in batches. This creates a dataloader that creates batches of 12 rows at once in tensor form.

In [9]:
from torch.utils.data import DataLoader
loader = DataLoader(dataset.text_tokens.values, batch_size=12)

# For debugging.
for i, batch in enumerate(loader):
        print(i, batch)
        break

0 tensor([[  101, 56898,   137,  ...,     0,     0,     0],
        [  101,   137, 74039,  ...,     0,     0,     0],
        [  101, 13229, 21885,  ...,     0,     0,     0],
        ...,
        [  101, 56898,   137,  ...,     0,     0,     0],
        [  101,   148, 20024,  ...,     0,     0,     0],
        [  101, 12399, 19122,  ...,     0,     0,     0]])


# Model 1: (distil)BERT

This model transform the list of ordered BERT id's in to a feature vector on which we can use regular classfiers (i.e. logistics classifiers, or kNN).

## Prepare model for distilBERT (if data is not preprocessed)

Make small batch set that easily fits in memory, with only necessary data included. It becomes a DataFrame of two columns: `text_token` which contains Numpy arrays of BERT ID's, and `like_timestamp` which contains `1` if liked and `0` if not liked.

### Create batch

In order to not run out of memory we have to work in small batches of the dataset at a time.

In [31]:
batch_1 = dataset[:2000]
batch_1 = batch_1[['text_tokens','like_timestamp']]

batch_1.text_tokens = batch_1.text_tokens.apply(lambda x: np.fromstring(x, dtype=int, sep="\t"))
batch_1.like_timestamp = batch_1.like_timestamp.apply(lambda x: 0 if math.isnan(x) else 1)

print(batch_1)

     text_tokens  like_timestamp
0             []               1
1             []               1
2             []               1
3             []               1
4             []               1
...          ...             ...
1995          []               1
1996          []               1
1997          []               1
1998          []               1
1999          []               1

[2000 rows x 2 columns]


### Optional

Get some stats about the batch

In [0]:
batch_1.like_timestamp.value_counts()

### Pad text tokens vector

In order for BERT model to work with the dataset we have to pad the input matrix rows to same size. So all `text_tokens` arrays have to be same length. Therefore we first calculate what the maximum vector length is in the `text_tokens` column

In [59]:
max_len = 0
for i in dataset.text_tokens.values:
  if len(i) > max_len:
      max_len = len(i)
      
print(max_len)

487


With the `max_len` we can padd all arrays in `text_tokens` to same length and export it to a 2d numpy array.

In [0]:
padded = np.array([np.concatenate([i, np.zeros(max_len-len(i), dtype=int)]) for i in batch_1.text_tokens.values])
print(padded)


In [12]:
np.array(padded).shape

(2000, 227)

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [13]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 227)

## Run model to get hidden states

Running the output yields a 768 length vector for each row in the dataset.

### Clean GPU memory

After running the model some things are left in the memory of the GPU this attempts to clean up as much as possible. Certainly not perfect.

In [0]:
# Clean GPU cache
if use_cuda:
  del input_ids, attention_mask, model
  torch.cuda.empty_cache()


### Load tensors and Run

This creates the tensors from input data (and sends them to GPU if available) and runs the model.

In [0]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

# Put everything on Cuda if available
if use_cuda:
  input_ids = input_ids.to(device)
  attention_mask = attention_mask.to(device)
  model.cuda()

with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask=attention_mask)

## Run model on DataLoader (automatically batched)

In order to work on larger datasets we can work in batches.

In [29]:
features = None
i = 0
with torch.no_grad():
  if use_cuda:
    model.cuda()
  for input_tokens_batch in loader:
    maxlen = input_tokens_batch.size(1)
    mask = torch.arange(maxlen)[None, :] < input_tokens_batch[:, None]

    if use_cuda:
      input_tokens_batch, mask = input_tokens_batch.to(device), mask.to(device)

    last_hidden_states = model(input_tokens_batch, attention_mask=mask)
    last_features = last_hidden_states[0][:,0,:].to('cpu').numpy()
    features = np.concatenate((features, last_features)) if features is not None else last_features

print(len(features))

KeyboardInterrupt: ignored

In [30]:
print(len(features))

199080


### Get output from model

Next we have to get the classification features from the output of BERT to a proper matrix. For more details on this see: https://github.com/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb

In [15]:
if use_cuda:
  features = last_hidden_states[0][:,0,:].to('cpu').numpy()
else:
  features = last_hidden_states[0][:,0,:].numpy()
print(features)

[[ 0.17406213 -0.04339711 -0.18432969 ...  0.5506897   0.18670759
  -0.19857799]
 [-0.11964405 -0.26719034  0.14500038 ...  0.30625767  0.03193806
  -0.04966994]
 [ 0.10105897 -0.01485022  0.1649243  ...  0.2965125   0.28076914
   0.0342547 ]
 ...
 [ 0.13603929 -0.33101082  0.10854725 ...  0.39265656  0.3065402
  -0.26869738]
 [-0.0618199   0.24845442  0.43925264 ...  0.26208812  0.10677199
  -0.0713095 ]
 [-0.13499364 -0.0298886  -0.3979845  ...  0.21658844  0.2567397
  -0.04231678]]


Now that we have the feature set, it is good to construct our label set as well.

In [17]:
labels = dataset[:12].like_timestamp

0     0
1     1
2     1
3     0
4     0
5     0
6     0
7     0
8     1
9     0
10    0
11    0
Name: like_timestamp, dtype: int64


In [18]:
print(len(features))
print(len(labels))

12
12


# Model 2: Logistics classifier
We got our output from the BERT model we can now train our logistics classifier to actually classify tweet engagements.

First we split our training set up into train & test set.

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

Next train our Logistics Classifier

In [20]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# Evaluating classifier

Now that we have our trained classifier let's see how it performs

In [21]:
lr_clf.score(test_features, test_labels)

0.3333333333333333

Let's compare that to a dummy classifier

In [22]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.600 (+/- 0.75)


So we currently perform ~10% better than a dummy classifier, awesome.

# Other stuff (utilities)

Some random stuff that was useful before

## Downloading and uploading to GCP

Following code blocks define code to download and upload training data to the GCP storage bucket.

In [0]:
# Setup GCP credentials
project_id = 'positive-nuance-274811'
bucket_name = 'recsys-twitter-challenge-us-jku-quarantwits'

from google.colab import auth
auth.authenticate_user()

!gcloud config set project {project_id}

In [0]:
# Download training data
!gsutil cp gs://{bucket_name}/train_updated.tsv train_updated.tsv

In [0]:
# Upload partitioned training data to GCP

# !gsutil cp train.001.tsv gs://{bucket_name}/train.001.tsv
# !gsutil cp train.002.tsv gs://{bucket_name}/train.002.tsv
# !gsutil cp train.003.tsv gs://{bucket_name}/train.003.tsv
# !gsutil cp train.004.tsv gs://{bucket_name}/train.004.tsv
# !gsutil cp train.005.tsv gs://{bucket_name}/train.005.tsv
# !gsutil cp train.006.tsv gs://{bucket_name}/train.006.tsv

# !gsutil cp train_updated.tsv gs://{bucket_name}/train_updated.tsv

## Download deleted tweet & user ID's

Tweets & users can be removed over time and we have to remove the corresponding tweet engagements to adhere to GDPR. This piece of code downloads all these ID's so that they can be removed from our dataset in the next step.

In [0]:
# !wget -O tweetIDs_1.txt 'https://elasticbeanstalk-us-west-2-800068098556.s3-us-west-2.amazonaws.com/challenge-website/public_data/training/diffs/tsv_deleted_engaged_with_tweet_id/2020/03/01'
# !wget -O tweetIDs_2.txt 'https://elasticbeanstalk-us-west-2-800068098556.s3-us-west-2.amazonaws.com/challenge-website/public_data/training/diffs/tsv_deleted_engaged_with_tweet_id/2020/03/08'
# !wget -O tweetIDs_3.txt 'https://elasticbeanstalk-us-west-2-800068098556.s3-us-west-2.amazonaws.com/challenge-website/public_data/training/diffs/tsv_deleted_engaged_with_tweet_id/2020/03/15'
# !wget -O tweetIDs_4.txt 'https://elasticbeanstalk-us-west-2-800068098556.s3-us-west-2.amazonaws.com/challenge-website/public_data/training/diffs/tsv_deleted_engaged_with_tweet_id/2020/03/22'
# !wget -O tweetIDs_5.txt 'https://elasticbeanstalk-us-west-2-800068098556.s3-us-west-2.amazonaws.com/challenge-website/public_data/training/diffs/tsv_deleted_engaged_with_tweet_id/2020/03/29'
# !wget -O tweetIDs_6.txt 'https://elasticbeanstalk-us-west-2-800068098556.s3-us-west-2.amazonaws.com/challenge-website/public_data/training/diffs/tsv_deleted_engaged_with_tweet_id/2020/04/12'

# !wget -O userIDs_1.txt 'https://elasticbeanstalk-us-west-2-800068098556.s3-us-west-2.amazonaws.com/challenge-website/public_data/training/diffs/tsv_deleted_user_id/2020/03/01'
# !wget -O userIDs_2.txt 'https://elasticbeanstalk-us-west-2-800068098556.s3-us-west-2.amazonaws.com/challenge-website/public_data/training/diffs/tsv_deleted_user_id/2020/03/08'
# !wget -O userIDs_3.txt 'https://elasticbeanstalk-us-west-2-800068098556.s3-us-west-2.amazonaws.com/challenge-website/public_data/training/diffs/tsv_deleted_user_id/2020/03/15'
# !wget -O userIDs_4.txt 'https://elasticbeanstalk-us-west-2-800068098556.s3-us-west-2.amazonaws.com/challenge-website/public_data/training/diffs/tsv_deleted_user_id/2020/03/22'
# !wget -O userIDs_5.txt 'https://elasticbeanstalk-us-west-2-800068098556.s3-us-west-2.amazonaws.com/challenge-website/public_data/training/diffs/tsv_deleted_user_id/2020/03/29'
# !wget -O userIDs_6.txt 'https://elasticbeanstalk-us-west-2-800068098556.s3-us-west-2.amazonaws.com/challenge-website/public_data/training/diffs/tsv_deleted_user_id/2020/04/12'

## Removing removed user & tweet ID's

After getting the removed ID's in the previous step this piece of code removes those ID's from the dataset and stores it as a new one. NOTE: this has to be run manually for each partition currently, by running it 6 times and changing the `00x` in the file names on lines `12` and `25` to the number of the partition.

In [0]:
all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",
                "tweet_type", "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",
                "engaged_with_user_following_count", "engaged_with_user_is_verified",
                "engaged_with_user_account_creation", "enaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count",
                "enaging_user_is_verified", "enaging_user_account_creation", "engagee_follows_engager",
                "reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]

new_dataset = pd.read_csv('./train.006.tsv', delimiter="\x01", encoding='utf-8', header=None)
new_dataset.columns = all_features

# print(new_dataset.tweet_id)
for i in range(1, 6):
  removed_tweet_ids = pd.read_csv("tweetIDs_{}.txt".format(i), header=None)
  new_dataset = new_dataset[~new_dataset.tweet_id.isin(removed_tweet_ids[0])]

for i in range(1, 6):
  removed_user_ids = pd.read_csv("userIDs_{}.txt".format(i), header=None)
  new_dataset = new_dataset[~new_dataset.engaged_with_user_id.isin(removed_user_ids[0])]
  new_dataset = new_dataset[~new_dataset.enaging_user_id.isin(removed_user_ids[0])]

new_dataset.to_csv('./train_updated.006.tsv', sep="\x01", header=False, index=False, quoting=csv.QUOTE_NONE)


## Unix tools used to split the dataset into partitions

In order to have more manageable dataset pieces, they are partitioned into sets of 10GB so that they can be read into memory.

In [0]:
# !chmod +x split.sh
# !apt update && apt install -y bc
# !./split.sh
# !wc -l train.001.tsv
cat train_updated.001.tsv train_updated.002.tsv train_updated.003.tsv train_updated.004.tsv train_updated.005.tsv train_updated.006.tsv > train_updated.tsv

## Dataset sampler

Can be used to create a random sample from a large training set. `sample_frequency` can be defined to what your heart desires.

In [0]:
sample_frequency = 1/6

sampled_set = dataset.sample(frac=sample_frequency, random_state=4173141592)
sampled_set.to_csv('./sample.tsv', sep="\x01", header=False, index=False, quoting=csv.QUOTE_NONE)


## Playground

Playground based on the code snippet by the RecSys challenge itself.
Can be used to efficiently read the first few lines of the dataset.

In [0]:
all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",
                "tweet_type", "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",
                "engaged_with_user_following_count", "engaged_with_user_is_verified",
                "engaged_with_user_account_creation", "enaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count",
                "enaging_user_is_verified", "enaging_user_account_creation", "engagee_follows_engager"]

all_features_to_idx = dict(zip(all_features, range(len(all_features))))
labels_to_idx = {"reply_timestamp": 20, "retweet_timestamp": 21, "retweet_with_comment_timestamp": 22,
                 "like_timestamp": 23}


def print_features(features):
  print(len(features))
  print(features)
  print('-----------------')
  for feature, idx in all_features_to_idx.items():
        print("feature {} has value {}".format(feature, features[idx]))

  for label, idx in labels_to_idx.items():
        print("label {} has value {}".format(label, features[idx]))

with open(train_file, encoding="utf-8") as fileobject:
    i = 0
    for line in fileobject:
        if i == 10:
            break
        line = line.strip()
        features = line.split("\x01")
        i += 1
        print_features(features)

        

In [0]:
dataset = sampled_set

## Vectorizing text_tokens

In [0]:
dataset['text_tokens'] = dataset['text_tokens'].str.split("\t")
print(dataset)

In [0]:
dataset['text_tokens'] = dataset['text_tokens'].apply(lambda x: np.fromstring(x, dtype=int, sep="\t"))